In [ ]:
!pip list

# < SQLAlchemy >
- 바인딩이 언제되고 연결이 언제되는지 알아야 한다.
- 엔진이 언제 생겼는지 
- 엔진에서 받아온 실제로 작업되는 과정을 추상화시켜서 만드는 것이다.
- 엔진을 접속하기 위해서는 메타 데이터로 감싼 데이터가 필요하다.
 1. engine create
 2. metadata create & table create & metadata는 engine으로 create
 3. engine connect
 4. insert
 5. select

In [1]:
import sqlalchemy 

In [5]:
sqlalchemy.__version__

'1.2.7'

### 1. create

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine("sqlite:///alchemy_core.db", echo = True) #memory, 없어도 상관 없다. 엔진을 통해서 sql 문을 생성만 해준다.

In [12]:
print(engine) #db를 memory에서 한다는 것을 의미한다. == main 함수 하나 만든 것을 의미한다. 

Engine(sqlite://)


In [4]:
from sqlalchemy import MetaData

In [5]:
metadata = MetaData()
print(metadata) #Metadata에 저장(biding) -> sql

MetaData(bind=None)


In [6]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey

In [16]:
metadata = MetaData()
users = Table('users', metadata,
             Column('id', Integer, primary_key = True),
             Column('name', String),
             Column('fullname', String))

In [17]:
address = Table('address',metadata,
               Column('id', Integer, primary_key = True),
               Column('user_id',Integer, ForeignKey('users.id')),
               Column('email', String, nullable = False))

In [18]:
metadata.create_all(engine) #실제 SQLite와 통신을 위해서 bind한다. echo=True해서 모든 로그 값이 나온다.

2018-07-10 13:42:20,568 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 13:42:20,568 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:42:20,570 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 13:42:20,570 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:42:20,572 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 13:42:20,572 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:42:20,574 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("address")
2018-07-10 13:42:20,574 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:42:20,576 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-07-10 13:42:20,578 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:42:20,579 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 13:42:20,580 INFO sqlalchemy.engine.bas

In [20]:
for row in metadata.tables: #테이블 2개 생성 완료
    print(row)

users
address


In [21]:
print(metadata)

MetaData(bind=None)


### 2. insert 

In [23]:
print(users.insert()) #이러한 Sql 언어를 추상화한 것이다. 

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [53]:
users.insert().values(name = 'kim', fullname = 'Anonymous, kim')

### 3. compile

In [62]:
insert = users.insert().values(name = 'kim', fullname = 'Anonymous, kim')

In [63]:
insert.compile().params #파라미터의 값들을 찾을 수 있다.

{'name': 'kim', 'fullname': 'Anonymous, kim'}

In [42]:
insert.bind = engine

NameError: name 'insert' is not defined

### 4. Pool
- engine 으로 바인딩된 것을 execute로 sqlite로 넘긴다.

In [8]:
conn = engine.connect()

2018-07-10 16:24:36,421 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:24:36,422 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:36,425 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:24:36,425 INFO sqlalchemy.engine.base.Engine ()


In [65]:
result = conn.execute(insert)

2018-07-10 14:27:40,381 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:27:40,381 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, kim')
2018-07-10 14:27:40,383 INFO sqlalchemy.engine.base.Engine COMMIT


In [33]:
result.inserted_primary_key #데이터를 몇 개 넣었는지 확인한다. 

[2]

In [66]:
insert = users.insert() #어디에 insert해야하는지 알고 있기 때문에 우리는 그냥 파라미터만 넣어주면 된다. 

In [67]:
conn.execute(insert,name = 'Lee', fullname = 'Unknown, Lee') #어느 테이블에서 실행해야하는지 알고 있다. 

2018-07-10 14:27:45,749 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:27:45,750 INFO sqlalchemy.engine.base.Engine ('Lee', 'Unknown, Lee')
2018-07-10 14:27:45,751 INFO sqlalchemy.engine.base.Engine COMMIT


In [68]:
result.inserted_primary_key

[7]

In [69]:
result = conn.execute(insert, name = 'xxx', fullname = 'xxxx, xxx')

2018-07-10 14:27:53,559 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:27:53,560 INFO sqlalchemy.engine.base.Engine ('xxx', 'xxxx, xxx')
2018-07-10 14:27:53,561 INFO sqlalchemy.engine.base.Engine COMMIT


In [70]:
result.inserted_primary_key

[9]

In [40]:
conn.execute(address.insert(), [
    {'user_id':1, 'email':'anonymous.kim@test.com'},
    {'user_id':2, 'email':'anonympous.Lee@test.com'}
])

2018-07-10 14:16:04,545 INFO sqlalchemy.engine.base.Engine INSERT INTO address (user_id, email) VALUES (?, ?)
2018-07-10 14:16:04,547 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'anonympous.Lee@test.com'))
2018-07-10 14:16:04,548 INFO sqlalchemy.engine.base.Engine COMMIT


### Select

In [15]:
from sqlalchemy.sql import select

In [72]:
query = select([users])
result = conn.execute(query)

for row in result:
    print(row)

2018-07-10 14:28:03,493 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:28:03,494 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, kim')
(2, 'kim', 'Anonymous, kim')
(3, 'Lee', 'Unknown, Lee')
(4, 'xxx', 'xxxx, xxx')
(5, 'Lee', 'Unknown, Lee')
(6, 'xxx', 'xxxx, xxx')
(7, 'kim', 'Anonymous, kim')
(8, 'Lee', 'Unknown, Lee')
(9, 'xxx', 'xxxx, xxx')


In [73]:
result = conn.execute(select([users.c.name, users.c.fullname]))

for row in result:
    print(row)

2018-07-10 14:28:10,469 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-07-10 14:28:10,470 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Anonymous, kim')
('kim', 'Anonymous, kim')
('Lee', 'Unknown, Lee')
('xxx', 'xxxx, xxx')
('Lee', 'Unknown, Lee')
('xxx', 'xxxx, xxx')
('kim', 'Anonymous, kim')
('Lee', 'Unknown, Lee')
('xxx', 'xxxx, xxx')


In [74]:
row = result.fetchone() #None일때까지 계속 fetch 한다. 
print(row) 

None


# Fetch는 값이 사라지는 것이 아니라 cursor의 위치를 변경

In [76]:
result = conn.execute(query) #cursor의 위치를 변경하는 것이다. fetch는 값이 사라지는 것이 아니다!
rows = result.fetchall()
for row in rows:
    print(row)

2018-07-10 14:29:10,836 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:29:10,837 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, kim')
(2, 'kim', 'Anonymous, kim')
(3, 'Lee', 'Unknown, Lee')
(4, 'xxx', 'xxxx, xxx')
(5, 'Lee', 'Unknown, Lee')
(6, 'xxx', 'xxxx, xxx')
(7, 'kim', 'Anonymous, kim')
(8, 'Lee', 'Unknown, Lee')
(9, 'xxx', 'xxxx, xxx')


# where
- select 문 뒤에 where 절을 쓴다.

In [17]:
from sqlalchemy import and_, or_, not_

In [79]:
print(users.c.id == address.c.user_id)

users.id = address.user_id


In [84]:
print(or_(users.c.id == address.c.user_id, users.c.id == 1)) #or 함수가 어떻게 실행되는지 확인 가능

users.id = address.user_id OR users.id = :id_1


In [85]:
print((users.c.id == address.c.user_id) | (users.c.id == 1)) #파이썬 문법으로도 가능하다. 

users.id = address.user_id OR users.id = :id_1


In [87]:
result = conn.execute(select([users]).where(users.c.id == 1))

2018-07-10 14:38:11,973 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2018-07-10 14:38:11,974 INFO sqlalchemy.engine.base.Engine (1,)


In [88]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim')


# where 절을 통한 Join

In [90]:
result = conn.execute(select([users, address]).where(users.c.id == address.c.user_id))

2018-07-10 14:42:37,062 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, address.id, address.user_id, address.email 
FROM users, address 
WHERE users.id = address.user_id
2018-07-10 14:42:37,063 INFO sqlalchemy.engine.base.Engine ()


In [91]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim', 1, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, kim', 2, 2, 'anonympous.Lee@test.com')


# Like

In [92]:
result = conn.execute(select([users]).where(users.c.name.like('k%')))

2018-07-10 14:45:18,972 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 14:45:18,973 INFO sqlalchemy.engine.base.Engine ('k%',)


In [93]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim')
(2, 'kim', 'Anonymous, kim')
(7, 'kim', 'Anonymous, kim')


In [94]:
result = conn.execute(select([users]).where(users.c.name.like('%i%')))

2018-07-10 14:45:56,776 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 14:45:56,776 INFO sqlalchemy.engine.base.Engine ('%i%',)


In [95]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim')
(2, 'kim', 'Anonymous, kim')
(7, 'kim', 'Anonymous, kim')


- and 절로 묶지 않아도 where절로 condition을 만들 수 있다.
- or는 따로 condition을 만들어줘야 한다.

In [98]:
result = conn.execute(select([users, address]).where(users.c.id == address.c.user_id).where(users.c.name.like('k%')))

2018-07-10 14:47:49,708 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, address.id, address.user_id, address.email 
FROM users, address 
WHERE users.id = address.user_id AND users.name LIKE ?
2018-07-10 14:47:49,709 INFO sqlalchemy.engine.base.Engine ('k%',)


In [99]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim', 1, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, kim', 2, 2, 'anonympous.Lee@test.com')


# Join

In [100]:
from sqlalchemy import join

In [103]:
print(users.join(address)) #foreign key를 명시했기 때문에 알아서 join이 된 것이다. 

users JOIN address ON users.id = address.user_id


### Inner Join

In [104]:
result = conn.execute(select([users]).select_from(users.join(address))) 

2018-07-10 15:10:12,099 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users JOIN address ON users.id = address.user_id
2018-07-10 15:10:12,099 INFO sqlalchemy.engine.base.Engine ()


In [105]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, kim')
(2, 'kim', 'Anonymous, kim')


In [107]:
result = conn.execute(select([users.c.name, address.c.email]).select_from(users.join(address))) 

2018-07-10 15:12:35,667 INFO sqlalchemy.engine.base.Engine SELECT users.name, address.email 
FROM users JOIN address ON users.id = address.user_id
2018-07-10 15:12:35,668 INFO sqlalchemy.engine.base.Engine ()


In [108]:
for row in result:
    print(row) 

('kim', 'anonymous.kim@test.com')
('kim', 'anonympous.Lee@test.com')


# 예제

### Create

In [9]:
artist = Table("Artist",metadata,
               Column("id", Integer, primary_key = True),
              Column("name", String, nullable = False),
              extend_existing = True)
album = Table("Album", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable = False),
             Column("artist_id", Integer, ForeignKey("Artist.id")),
             extend_existing = True)
genre = Table("Genre",metadata,
               Column("id", Integer, primary_key = True),
              Column("name", String, nullable = False),
              extend_existing = True)
track = Table("Track", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable = False),
             Column("length", Integer, nullable = False),
             Column("rating", Integer, nullable = False),
             Column("count", Integer, nullable = False),
             Column("album_id", Integer, ForeignKey("Album.id")),
             Column("genre_id", Integer, ForeignKey("Genre.id")),
             extend_existing = True)

### Show Tables

In [10]:
metadata.create_all(engine)

2018-07-10 16:24:56,330 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2018-07-10 16:24:56,331 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:56,333 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 16:24:56,334 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:56,335 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2018-07-10 16:24:56,335 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:56,336 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2018-07-10 16:24:56,337 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:56,338 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2018-07-10 16:24:56,338 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:24:56,340 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:24:56,341 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Genre" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT N

In [11]:
for row in metadata.tables:
    print(row)

Artist
Album
Genre
Track


In [128]:
users.drop(engine)

2018-07-10 15:32:34,977 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users
2018-07-10 15:32:34,978 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:32:34,980 INFO sqlalchemy.engine.base.Engine COMMIT


In [129]:
address.drop(engine)

2018-07-10 15:32:36,601 INFO sqlalchemy.engine.base.Engine 
DROP TABLE address
2018-07-10 15:32:36,603 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:32:36,604 INFO sqlalchemy.engine.base.Engine COMMIT


In [130]:
for row in metadata.tables:
    print(row)

users
address
Artist
Album
Genre
Track


In [131]:
for row in engine.table_names():
    print(row)

2018-07-10 15:32:39,384 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 15:32:39,385 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


### Insert

In [12]:
conn.execute(artist.insert(),[
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])
conn.execute(album.insert(),[
    {"title":"IV","artist_id":1},
    {"title":"Who Made Who", "artist_id":2}
])
conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Metal"}
])
conn.execute(track.insert(),[
    {"title":"Black Dog", "rating":5, "length":297,"count":0,"album_id":1, "genre_id":1},
    {"title":"Stairway", "rating":5, "length":482,"count":0,"album_id":1, "genre_id":2},
    {"title":"Acout to rock", "rating":5, "length":313,"count":0,"album_id":2, "genre_id":1},
    {"title":"Who Made Who", "rating":5, "length":297,"count":0,"album_id":2, "genre_id":2},
])

2018-07-10 16:25:10,949 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2018-07-10 16:25:10,950 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2018-07-10 16:25:10,952 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:25:10,953 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2018-07-10 16:25:10,954 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2018-07-10 16:25:10,955 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:25:10,957 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2018-07-10 16:25:10,957 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2018-07-10 16:25:10,959 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:25:10,960 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2018-07-10 16:25:10,961 INFO sqlalchemy.engine.base.Engine (('Black Dog', 

### Select

In [13]:
artistResult = conn.execute(artist.select())
for row in artistResult:
    print(row)
    
albumResult = conn.execute(album.select())
for row in albumResult:
    print(row)
    
genreResult = conn.execute(genre.select())
for row in genreResult:
    print(row)

trackResult = conn.execute(track.select())
for row in trackResult:
    print(row)

2018-07-10 16:25:15,226 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2018-07-10 16:25:15,227 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led Zepplin')
(2, 'AC/DC')
2018-07-10 16:25:15,230 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-10 16:25:15,231 INFO sqlalchemy.engine.base.Engine ()
(1, 'IV', 1)
(2, 'Who Made Who', 2)
2018-07-10 16:25:15,233 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id, "Genre".name 
FROM "Genre"
2018-07-10 16:25:15,233 INFO sqlalchemy.engine.base.Engine ()
(1, 'Rock')
(2, 'Metal')
2018-07-10 16:25:15,235 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2018-07-10 16:25:15,236 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)
(3, 'Acout to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5,

### Where

In [18]:
result = conn.execute(select([track])
                     .where(
                         and_(track.c.album_id == 1,
                             or_(
                                 track.c.genre_id == 1,
                                 track.c.genre_id == 2)))
                     )
for row in result:
    print(row)

2018-07-10 16:25:56,729 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 16:25:56,731 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)


### Update

In [19]:
from sqlalchemy import update

conn.execute(track.update().values(genre_id = 3).where(track.c.id == 1))

2018-07-10 16:26:01,630 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 16:26:01,632 INFO sqlalchemy.engine.base.Engine (3, 1)
2018-07-10 16:26:01,634 INFO sqlalchemy.engine.base.Engine COMMIT


In [20]:
result = conn.execute(select([track])
                     .where(
                         and_(track.c.album_id == 1,
                             or_(
                                 track.c.genre_id == 1,
                                 track.c.genre_id == 2)))
                     )
for row in result:
    print(row)

2018-07-10 16:26:04,441 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 16:26:04,442 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(2, 'Stairway', 482, 5, 0, 1, 2)


In [21]:
from sqlalchemy import update

conn.execute(track.update().values(genre_id = 1).where(track.c.id == 3))

2018-07-10 16:26:07,543 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 16:26:07,545 INFO sqlalchemy.engine.base.Engine (1, 3)
2018-07-10 16:26:07,546 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
result = conn.execute(select([track])
                     .where(
                         and_(track.c.album_id == 1,
                             or_(
                                 track.c.genre_id == 1,
                                 track.c.genre_id == 2)))
                     )
for row in result:
    print(row)

2018-07-10 16:26:20,196 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 16:26:20,197 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(2, 'Stairway', 482, 5, 0, 1, 2)


### Join

In [145]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [23]:
result = conn.execute(track.select().select_from(track.join(album)))

for row in result.fetchall():
    print(row)

2018-07-10 16:26:25,171 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 16:26:25,172 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 3)
(2, 'Stairway', 482, 5, 0, 1, 2)
(3, 'Acout to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [24]:
result = conn.execute(track
                     .select()
                     .select_from(track.join(album))
                     .where(album.c.id == 1))

for row in result.fetchall():
    print(row)

2018-07-10 16:26:27,467 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id 
WHERE "Album".id = ?
2018-07-10 16:26:27,468 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'Black Dog', 297, 5, 0, 1, 3)
(2, 'Stairway', 482, 5, 0, 1, 2)


### Multiple Join

In [25]:
print(track.join(album))
print(track.join(album).join(genre))
print(track.join(album).join(genre).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [26]:
result = conn.execute(select([track.c.title,album.c.title, genre.c.name, artist.c.name])
                     .select_from(track.join(album).join(genre).join(artist)))

for row in result:
    print(row)

2018-07-10 16:26:32,227 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2018-07-10 16:26:32,228 INFO sqlalchemy.engine.base.Engine ()
('Stairway', 'IV', 'Metal', 'Led Zepplin')
('Acout to rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


In [27]:
result = conn.execute(select([track.c.title,album.c.title, genre.c.name, artist.c.name])
                     .select_from(track.join(album).join(genre).join(artist))
                     .where(
                         and_(
                         genre.c.id == 1,
                         artist.c.id == 1)))

for row in result:
    print(row)

2018-07-10 16:26:36,997 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".id = ?
2018-07-10 16:26:36,998 INFO sqlalchemy.engine.base.Engine (1, 1)


In [28]:
conn.close()
metadata.clear()

### Open/Close

In [40]:
from sqlalchemy import create_engine, MetaData

engine = create_engine("sqlite:///alchemy_core.db", echo = True)
conn = engine.connect()

metadata = MetaData(bind = engine, reflect = True)
metadata.reflect(bind = engine)

2018-07-10 16:36:13,585 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:36:13,585 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:36:13,587 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:36:13,588 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:36:13,590 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 16:36:13,590 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:36:13,591 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 16:36:13,592 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:36:13,593 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-10 16:36:13,593 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:36:13,594 INFO sqlalchemy.engine.base.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [41]:
for row in metadata.tables:
    print(row)
    #metadata에는 engine에서 가져온 데이터들이 존재하고 있음 

Album
Artist
Genre
Track


In [35]:
album = metadata.tables["Album"]

In [36]:
result = conn.execute(album.select())

2018-07-10 16:33:10,356 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-10 16:33:10,357 INFO sqlalchemy.engine.base.Engine ()


In [37]:
for row in result:
    print(row)

(1, 'IV', 1)
(2, 'Who Made Who', 2)


In [38]:
tables = metadata.tables
for table in tables:
    print(table)
    
track = metadata.tables["Track"]
track 

for row in conn.execute(track.select()).fetchall():
    print(row)

conn.close()
metadata.clear()

Album
Artist
Genre
Track
2018-07-10 16:33:33,593 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2018-07-10 16:33:33,594 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 3)
(2, 'Stairway', 482, 5, 0, 1, 2)
(3, 'Acout to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


# < ORM >
- object (class) 와 Database을 mapping

## Declare
- metadata -> table
- base -> class

# 0712 실습 과정!
1. engine 생성
2. base 생성
3. relationship import
4. 각 class 생성
5. metedata를 생성
6. 값을 넣고 artist1은 그 row를 가리키는 포인터이다. (artist1 = Artist(name = "Led Zepplin"))
7. 마찬가지로 album을 향해서 값을 변경할 수 있다.
8. 세션을 켜서 engine과 bind를 한다.
9. 그 다음에 포인터를 세션에 bind를 해준다.
10. commit을 하면 그 값이 DB에 저장된다. 

In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///0712.db", echo = True)

In [2]:
from sqlalchemy.ext.declarative import declarative_base 
base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey

In [4]:
from sqlalchemy.orm import relationship

In [5]:
class Artist(base):
    __tablename__ = 'artist'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    album = relationship("Album", back_populates = "artist")

In [6]:
class Album(base):
    __tablename__ = 'album'
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    artist = relationship("Artist", back_populates = "album")

In [7]:
class Genre(base):
    __tablename__ = 'genre'
    id = Column(Integer, primary_key = True)
    name = Column(String)

class Track(base):
    __tablename__ = 'track'
    id = Column(Integer, primary_key = True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))

In [8]:
base.metadata.create_all(engine)

2018-07-12 10:55:26,738 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-12 10:55:26,739 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:55:26,741 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-12 10:55:26,742 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:55:26,743 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-12 10:55:26,744 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:55:26,746 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-12 10:55:26,746 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:55:26,747 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-12 10:55:26,748 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:55:26,749 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-12 10:55:26,749 INFO sqlalchemy.engine.base.Engine ()


In [9]:
artist1 = Artist(name = 'Led Zepplin')
artist2 = Artist(name = 'AC/DC')

In [10]:
artist1 #self 어쩌구를 넣으면 값을 나오도록 만들 수 있다. 

In [11]:
artist1.album = [Album(title = "IV"), Album(title = "Who made Who")]
#commit해서 DB알려주기 전까지는 그냥 engine으로 하는 것 

In [12]:
from sqlalchemy.orm import sessionmaker

In [13]:
Session = sessionmaker(bind = engine)

In [15]:
session = Session()

In [17]:
session.add(artist1) #bind후 
session.commit() #DB에 저장 

2018-07-12 10:56:18,380 INFO sqlalchemy.engine.base.Engine UPDATE album SET title=? WHERE album.id = ?
2018-07-12 10:56:18,381 INFO sqlalchemy.engine.base.Engine ('test', 1)
2018-07-12 10:56:18,383 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
print(artist1.id, artist1.name, artist1.album[0].id)
#artist1.album은 리스트 형태
artist1.album[0].title = "test" #이렇게 변경이 가능하다. 

2018-07-12 10:56:26,259 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 10:56:26,260 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-12 10:56:26,261 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-12 10:56:26,263 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-12 10:56:26,263 INFO sqlalchemy.engine.base.Engine (1,)
1 Led Zepplin 1


In [20]:
session.add_all([artist1, artist2])

In [21]:
session.commit()

2018-07-10 17:21:11,696 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:21:11,698 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name, fullname, passwd) VALUES (?, ?, ?)
2018-07-10 17:21:11,698 INFO sqlalchemy.engine.base.Engine ('Led Zepplin', None, None)
2018-07-10 17:21:11,699 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name, fullname, passwd) VALUES (?, ?, ?)
2018-07-10 17:21:11,700 INFO sqlalchemy.engine.base.Engine ('AC/DC', None, None)
2018-07-10 17:21:11,701 INFO sqlalchemy.engine.base.Engine COMMIT


## dirty

In [22]:
session.dirty #instance변경 사항에 대한 내용을 알려준다. 

IdentitySet([])

- 아직 메모리에 존재하고 DB에는 존재하지 않은 상

In [23]:
session.new

IdentitySet([])

In [24]:
artist1.name = 'xyz'

In [25]:
session.dirty

2018-07-10 17:23:37,036 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:23:37,038 INFO sqlalchemy.engine.base.Engine SELECT artist.passwd AS artist_passwd, artist.id AS artist_id, artist.fullname AS artist_fullname 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:23:37,038 INFO sqlalchemy.engine.base.Engine (1,)


IdentitySet([<T'User(name = 'xyz', fullname= 'None', password= 'None')>])

In [26]:
session.commit()

2018-07-10 17:24:07,998 INFO sqlalchemy.engine.base.Engine UPDATE artist SET name=? WHERE artist.id = ?
2018-07-10 17:24:07,999 INFO sqlalchemy.engine.base.Engine ('xyz', 1)
2018-07-10 17:24:08,001 INFO sqlalchemy.engine.base.Engine COMMIT


In [27]:
session.dirty #commit을 하고나면 변경된 내용이 없어지게 된다. 

IdentitySet([])

In [30]:
session.add_all([
    Album(title = '1', artist_id = 1),
    Album(title = '2', artist_id = 2)
])

In [31]:
session.new

IdentitySet([<__main__.Album object at 0x10bf562e8>, <__main__.Album object at 0x10bf56358>])

### commit
- DB에 저장

In [33]:
session.commit()

2018-07-10 17:26:21,727 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:26:21,729 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-10 17:26:21,729 INFO sqlalchemy.engine.base.Engine ('1', 1)
2018-07-10 17:26:21,731 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-10 17:26:21,732 INFO sqlalchemy.engine.base.Engine ('2', 2)
2018-07-10 17:26:21,733 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
session.add_all([
    Genre(name = "Rock"),
    Genre(name = "Metal")
])

In [35]:
session.add_all([
    Track(title = '1-1', album_id = 1, genre_id = 1),
    Track(title = '1-2', album_id = 1, genre_id = 2),
    Track(title = '2-1', album_id = 2, genre_id = 1),
    Track(title = '2-2', album_id = 2, genre_id = 2)
])

In [36]:
session.new

IdentitySet([<__main__.Genre object at 0x10bf56a90>, <__main__.Genre object at 0x10bf56c18>, <__main__.Track object at 0x10bf60320>, <__main__.Track object at 0x10bf60390>, <__main__.Track object at 0x10bf60400>, <__main__.Track object at 0x10bf60470>])

In [37]:
session.commit()

2018-07-10 17:28:24,423 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:28:24,425 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:28:24,426 INFO sqlalchemy.engine.base.Engine ('1-1', 1, 1)
2018-07-10 17:28:24,427 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:28:24,428 INFO sqlalchemy.engine.base.Engine ('1-2', 1, 2)
2018-07-10 17:28:24,428 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:28:24,429 INFO sqlalchemy.engine.base.Engine ('2-1', 2, 1)
2018-07-10 17:28:24,430 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:28:24,430 INFO sqlalchemy.engine.base.Engine ('2-2', 2, 2)
2018-07-10 17:28:24,431 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-10 17:28:24,432 INFO sqlalchemy.engine.ba

In [38]:
temp = Track(title = '3-1', album_id = 3, genre_id = 3)

In [40]:
print(temp.id)

None


In [41]:
session.add(temp)

In [42]:
session.new #database에 기록이 안 된 상태

IdentitySet([<__main__.Track object at 0x10bf56f60>])

In [43]:
session.commit()

2018-07-10 17:31:24,261 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:31:24,262 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:31:24,263 INFO sqlalchemy.engine.base.Engine ('3-1', 3, 3)
2018-07-10 17:31:24,264 INFO sqlalchemy.engine.base.Engine COMMIT


In [44]:
print(temp.id) #쉽게 데이터를 가져올 수 있는 것이다. 

2018-07-10 17:31:34,652 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:31:34,654 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-07-10 17:31:34,655 INFO sqlalchemy.engine.base.Engine (5,)
5


In [46]:
for row in session.query(Artist):
    print(row)

2018-07-10 17:33:28,206 INFO sqlalchemy.engine.base.Engine SELECT artist.passwd AS artist_passwd, artist.id AS artist_id, artist.name AS artist_name, artist.fullname AS artist_fullname 
FROM artist
2018-07-10 17:33:28,206 INFO sqlalchemy.engine.base.Engine ()
<T'User(name = 'xyz', fullname= 'None', password= 'None')>
<T'User(name = 'AC/DC', fullname= 'None', password= 'None')>


In [47]:
for row in session.query(Artist):
    print(row.name)

2018-07-10 17:33:46,614 INFO sqlalchemy.engine.base.Engine SELECT artist.passwd AS artist_passwd, artist.id AS artist_id, artist.name AS artist_name, artist.fullname AS artist_fullname 
FROM artist
2018-07-10 17:33:46,615 INFO sqlalchemy.engine.base.Engine ()
xyz
AC/DC


In [49]:
for row in session.query(Artist).filter(Artist.id == 1):
    print(row.name)

2018-07-10 17:35:35,736 INFO sqlalchemy.engine.base.Engine SELECT artist.passwd AS artist_passwd, artist.id AS artist_id, artist.name AS artist_name, artist.fullname AS artist_fullname 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:35:35,737 INFO sqlalchemy.engine.base.Engine (1,)
xyz
